## Import essential packages

In [1]:
#----- These are for the simulation 

from __future__ import division
import hoomd
import hoomd.md
import numpy
from matplotlib import pyplot

#----- These are for the orientation analysis

import gsd
import gsd.hoomd

# Simulation

In [2]:
#-----Some important variables

box = 12.5
replicates = 10
p_max = 0.4
t1 = 1.0
t2 = 1.2
t3 = 1.4
t4 = 1.6
t5 = 1.8
t6 = 2.0

In [3]:
#-------Define the unit cell

hoomd.context.initialize("");

uc = hoomd.lattice.unitcell(N = 1,
                            a1 = [box, 0, 0],
                            a2 = [0, box, 0],
                            a3 = [0, 0, box],
                            dimensions = 3,
                            position = [[0,0,0]],
                            type_name = ['M'],
                            mass = [1.0],
                            moment_inertia = [[0,
                                               1/12*1.0*8**2,
                                               1/12*1.0*8**2]],
                            orientation = [[1, 0, 0, 0]]);

#-----From the lattice we obtain a snapshot and initialize.

snapshot = uc.get_snapshot()
snapshot.replicate(replicates,replicates,replicates)
system = hoomd.init.read_snapshot(snapshot)

#-----Add the particle type for the constituent particles.

system.particles.types.add('A');

#-----Define each rigid body type in the local coordinate system of the body.

rigid = hoomd.md.constrain.rigid();
rigid.set_param('M', 
                types=['A']*8,
                positions=[(-4,0,0),(-3,0,0),(-2,0,0),(-1,0,0),
                           (1,0,0),(2,0,0),(3,0,0),(4,0,0)]);

#-----Instruct the rigid constraint to create the constituent particles

rigid.create_bodies();

#-----Define the potential energy

nl = hoomd.md.nlist.cell()

lj = hoomd.md.pair.lj(r_cut = 3.5, nlist=nl)
lj.set_params(mode='shift')

#-----Define interaction with species in the binary mixture

lj.pair_coeff.set('M','M', epsilon = 1.0, sigma = 1.0)
lj.pair_coeff.set('A','A', epsilon = 1.0, sigma = 1.0)
lj.pair_coeff.set('M','A', epsilon = 1.0, sigma = 1.0)


#-----Select an standard integrator

hoomd.md.integrate.mode_standard(dt=0.005);

#------Define group

mesogens = hoomd.group.rigid_center();

#----Integrate using NPT

pressure = hoomd.variant.linear_interp(points = [(0,1e-6), (1e4, p_max)])

npt = hoomd.md.integrate.npt(group = mesogens, kT = t2, tau = 0.5, tauP = 1.0, P = pressure);
npt.randomize_velocities(seed=42)

HOOMD-blue 2.6.0 DOUBLE HPMC_MIXED TBB SSE SSE2 SSE3 
Compiled: 05/29/2019
Copyright (c) 2009-2019 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, C D Lorenz, and A Travesset. "General purpose molecular dynamics
  simulations fully implemented on graphics processing units", Journal of
  Computational Physics 227 (2008) 5342--5359
* J Glaser, T D Nguyen, J A Anderson, P Liu, F Spiga, J A Millan, D C Morse, and
  S C Glotzer. "Strong scaling of general-purpose molecular dynamics simulations
  on GPUs", Computer Physics Communications 192 (2015) 97--107
-----
HOOMD-blue is running on the CPU
notice(2): Group "all" created containing 1000 particles
notice(2): constrain.rigid(): Creating 1000 rigid bodies (adding 8000 particles)
notice(2): Group "rigid_center" created containing 1000 particles


In [4]:
#-----Write output and Run the Simulation

log_file = "trayectory_T_" + str(t2) + "_P_" + str(p_max) + ".log"
gsd_file = "trayectory_T_" + str(t2) + "_P_" + str(p_max) + ".gsd"  
log = hoomd.analyze.log(filename = log_file,
                         quantities = ['num_particles',
                                     'ndof',
                                     'translational_ndof',
                                     'rotational_ndof',
                                     'potential_energy',
                                     'kinetic_energy',
                                     'translational_kinetic_energy',
                                     'rotational_kinetic_energy',
                                     'temperature',
                                     'pressure',
                                      'volume',
                                      'npt_thermostat_energy'],
                         period = 1e3,
                         overwrite = True);
gsd = hoomd.dump.gsd(gsd_file,
               period = 1e3,
               group = hoomd.group.all(),
               overwrite=True); 
hoomd.run(1.1e5)

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 9000
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: yes
** starting run **
Time 00:00:10 | Step 565 / 110000 | TPS 56.4681 | ETA 00:32:17
Time 00:00:20 | Step 834 / 110000 | TPS 26.6741 | ETA 01:08:12
Time 00:00:30 | Step 1058 / 110000 | TPS 22.3762 | ETA 01:21:08
Time 00:00:40 | Step 1267 / 110000 | TPS 20.8077 | ETA 01:27:05
Time 00:00:50 | Step 1493 / 110000 | TPS 22.551 | ETA 01:20:11
Time 00:01:00 | Step 1712 / 110000 | TPS 21.8529 | ETA 01:22:35
Time 00:01:10 | Step 1921 / 110000 | TPS 20.777 | ETA 01:26:41
Time 00:01:20 | Step 2143 / 110000 | TPS 22.0632 | ETA 01:21:28
Time 00:01:30 | Step 2363 / 110000 | TPS 21.7228 | ETA 01:22:35
Time 00:01:40 | Step 2572 / 110000 | TPS 20.861 | ETA 01:25:49
Time 00:01:50 | Step 2787 / 110000 | TPS 21.4996 | ETA 01:23:06
Time 00:02:00 | Step 3001 / 110000 | TPS 21.2056 | ETA 01:

In [5]:
#-----Updating parameters and directories

npt.set_params(kT = t3)
log_file = "trayectory_T_" + str(t3) + "_P_" + str(p_max) + ".log"
gsd_file = "trayectory_T_" + str(t3) + "_P_" + str(p_max) + ".gsd"  
log = hoomd.analyze.log(filename = log_file,
                         quantities = ['num_particles',
                                     'ndof',
                                     'translational_ndof',
                                     'rotational_ndof',
                                     'potential_energy',
                                     'kinetic_energy',
                                     'translational_kinetic_energy',
                                     'rotational_kinetic_energy',
                                     'temperature',
                                     'pressure',
                                      'volume',
                                      'npt_thermostat_energy'],
                         period = 1e3,
                         overwrite = True);
gsd = hoomd.dump.gsd(gsd_file,
               period = 1e3,
               group = hoomd.group.all(),
               overwrite=True); 
hoomd.run(1e5)

** starting run **
Time 01:42:24 | Step 110154 / 210000 | TPS 15.295 | ETA 01:48:48


**ERROR**: Particle with unique tag 1800 has NaN for its position.


RuntimeError: Error computing cell list

In [ ]:
#-----Updating parameters and directories

npt.set_params(kT = t4)
log_file = "trayectory_T_" + str(t4) + "_P_" + str(p_max) + ".log"
gsd_file = "trayectory_T_" + str(t4) + "_P_" + str(p_max) + ".gsd" 
log = hoomd.analyze.log(filename = log_file,
                         quantities = ['num_particles',
                                     'ndof',
                                     'translational_ndof',
                                     'rotational_ndof',
                                     'potential_energy',
                                     'kinetic_energy',
                                     'translational_kinetic_energy',
                                     'rotational_kinetic_energy',
                                     'temperature',
                                     'pressure',
                                      'volume',
                                      'npt_thermostat_energy'],
                         period = 1e3,
                         overwrite = True);
gsd = hoomd.dump.gsd(gsd_file,
               period = 1e3,
               group = hoomd.group.all(),
               overwrite=True);  
hoomd.run(1e5)

In [ ]:
#-----Updating parameters and directories

npt.set_params(kT = t5)
log_file = "trayectory_T_" + str(t5) + "_P_" + str(p_max) + ".log"
gsd_file = "trayectory_T_" + str(t5) + "_P_" + str(p_max) + ".gsd" 
log = hoomd.analyze.log(filename = log_file,
                         quantities = ['num_particles',
                                     'ndof',
                                     'translational_ndof',
                                     'rotational_ndof',
                                     'potential_energy',
                                     'kinetic_energy',
                                     'translational_kinetic_energy',
                                     'rotational_kinetic_energy',
                                     'temperature',
                                     'pressure',
                                      'volume',
                                      'npt_thermostat_energy'],
                         period = 1e3,
                         overwrite = True);
gsd = hoomd.dump.gsd(gsd_file,
               period = 1e3,
               group = hoomd.group.all(),
               overwrite=True);  
hoomd.run(1e5)

In [ ]:
#-----Updating parameters and directories

npt.set_params(kT = t6)
log_file = "trayectory_T_" + str(t6) + "_P_" + str(p_max) + ".log"
gsd_file = "trayectory_T_" + str(t6) + "_P_" + str(p_max) + ".gsd" 
log = hoomd.analyze.log(filename = log_file,
                         quantities = ['num_particles',
                                     'ndof',
                                     'translational_ndof',
                                     'rotational_ndof',
                                     'potential_energy',
                                     'kinetic_energy',
                                     'translational_kinetic_energy',
                                     'rotational_kinetic_energy',
                                     'temperature',
                                     'pressure',
                                      'volume',
                                      'npt_thermostat_energy'],
                         period = 1e3,
                         overwrite = True);
gsd = hoomd.dump.gsd(gsd_file,
               period = 1e3,
               group = hoomd.group.all(),
               overwrite=True);  
hoomd.run(1e5)

## System initial density

In [ ]:
particles = snapshot.particles.N

lx = snapshot.box.Lx
ly = snapshot.box.Ly
lz = snapshot.box.Lz
volume = lx * ly *lz

rho = particles / volume

print("Density = ", rho)

In [ ]:
system.get_metadata()